In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("Regulatory Affairs of Road Accident Data 2020 India.csv")

In [3]:
df.head()

,Million Plus Cities,Cause category,Cause Subcategory,Outcome of Incident,Count
0,Agra,Traffic Control,Flashing Signal/Blinker,Greviously Injured,0.0
1,Agra,Traffic Control,Flashing Signal/Blinker,Minor Injury,0.0
2,Agra,Traffic Control,Flashing Signal/Blinker,Persons Killed,0.0
3,Agra,Traffic Control,Flashing Signal/Blinker,Total Injured,0.0
4,Agra,Traffic Control,Flashing Signal/Blinker,Total number of Accidents,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9550 entries, 0 to 9549
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Million Plus Cities  9550 non-null   object 
 1   Cause category       9550 non-null   object 
 2   Cause Subcategory    9550 non-null   object 
 3   Outcome of Incident  9550 non-null   object 
 4   Count                9547 non-null   float64
dtypes: float64(1), object(4)
memory usage: 373.2+ KB


In [5]:
df.shape


(9550, 5)

In [6]:
df.describe()

,Count
count,9547.000000
mean,103.627632
std,275.189303
min,0.000000
25%,0.000000
50%,12.000000
75%,70.500000
max,3148.000000


In [7]:
df.describe(include= "object")

,Million Plus Cities,Cause category,Cause Subcategory,Outcome of Incident
count,9550,9550,9550,9550
unique,50,6,35,5
top,Agra,Road Features,Others,Greviously Injured
freq,191,2000,1450,2000


In [8]:
df.isnull().sum()

,0
Million Plus Cities,0
Cause category,0
Cause Subcategory,0
Outcome of Incident,0
Count,3


In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
df["Count"] = df["Count"].fillna(0)

In [19]:

!pip install plotly

In [20]:
import pandas as pd
import plotly.express as px

In [21]:
city_data = df.groupby("Million Plus Cities")["Count"].sum().reset_index()

fig = px.bar(
    city_data,
    x="Million Plus Cities",
    y="Count",
    title="Accidents by City",
)

fig.show()

In [22]:
cause_data = df.groupby("Cause category")["Count"].sum().reset_index()

fig = px.pie(
    cause_data,
    names="Cause category",
    values="Count",
    title="Accident Cause Distribution"
)

fig.show()

In [23]:
outcome_data = df.groupby("Outcome of Incident")["Count"].sum().reset_index()

fig = px.bar(
    outcome_data,
    x="Outcome of Incident",
    y="Count",
    color="Outcome of Incident",
    title="Accident Outcome Distribution"
)

fig.show()

In [24]:
pivot = df.groupby(["Cause category", "Outcome of Incident"])["Count"].sum().reset_index()

fig = px.bar(
    pivot,
    x="Cause category",
    y="Count",
    color="Outcome of Incident",
    barmode="stack",
    title="Cause vs Outcome Relationship"
)

fig.show()

In [25]:
heatmap_data = df.pivot_table(
    values="Count",
    index="Cause category",
    columns="Outcome of Incident",
    aggfunc="sum"
)

fig = px.imshow(heatmap_data,
                text_auto=True,
                title="Cause vs Outcome Heatmap")

fig.show()

In [94]:
!pip install scikit-learn

In [95]:
import pandas as pd
import joblib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [96]:
df = pd.read_csv("Regulatory Affairs of Road Accident Data 2020 India.csv")

In [97]:
le_city = LabelEncoder()
le_cause = LabelEncoder()
le_sub = LabelEncoder()
le_outcome = LabelEncoder()

# Fit encoders on ORIGINAL TEXT
le_city.fit(df["Million Plus Cities"])
le_cause.fit(df["Cause category"])
le_sub.fit(df["Cause Subcategory"])
le_outcome.fit(df["Outcome of Incident"])

LabelEncoder()

In [98]:
df["Million Plus Cities"] = le_city.transform(df["Million Plus Cities"])
df["Cause category"] = le_cause.transform(df["Cause category"])
df["Cause Subcategory"] = le_sub.transform(df["Cause Subcategory"])
df["Outcome of Incident"] = le_outcome.transform(df["Outcome of Incident"])

In [99]:
df["Accident_Risk"] = df["Count"] / df["Count"].max()
df["City_Accident_Frequency"] = df.groupby("Million Plus Cities")["Count"].transform("sum")
df["Cause_Severity"] = df.groupby("Cause category")["Count"].transform("mean")

In [100]:
X = df[
    [
        "Count",
        "Cause category",
        "Cause Subcategory",
        "Million Plus Cities",
        "Accident_Risk",
        "City_Accident_Frequency",
        "Cause_Severity"
    ]
]

y = df["Outcome of Incident"]

In [101]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [103]:
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300)

In [104]:
joblib.dump(model, "accident_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le_city, "city_encoder.pkl")
joblib.dump(le_cause, "cause_encoder.pkl")
joblib.dump(le_sub, "sub_encoder.pkl")
joblib.dump(le_outcome, "outcome_encoder.pkl")

['outcome_encoder.pkl']

In [105]:
from google.colab import files

files.download("accident_model.pkl")
files.download("scaler.pkl")
files.download("city_encoder.pkl")
files.download("cause_encoder.pkl")
files.download("sub_encoder.pkl")
files.download("outcome_encoder.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>